In [ ]:
from policy import *

newGame = [1, 3, 5, 7]
state = [1, 3, 5, 7]

win = (0, 0, 0, 0)

# Reward values for win and loss conditions
win_reward = 100
lose_reward = -50

# episode length
maxrun = 100

In [ ]:
policy = Policy_Template.copy()
value = Value_Template.copy()
q = Q_Template.copy()

In [ ]:
def Initialize():
    policy = Policy_Template.copy()
    value = Value_Template.copy()
    q = Q_Template.copy()
    return


def New_Game():
    state = newGame.copy()
    return


In [ ]:
"""
A function for setting the policy for a given state
r1: Number of sticks in row 1
r2: Number of sticks in row 2
r3: Number of sticks in row 3
r4: Number of sticks in row 4
count: Total number of sticks left
"""
def setPolicy(r1, r2, r3, r4, count):
    policy = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    for i in range(r1):
        policy[0 + i] = 1.0/count
    for i in range(r2):
        policy[1 + i] = 1.0/count
    for i in range(r3):
        policy[4 + i] = 1.0/count
    for i in range(r4):
        policy[9 + i] = 1.0/count
    return policy

"""
Generates a new policy for NIM
"""
def newPolicy():
    for r1 in range(2):
        for r2 in range(4):
            for r3 in range(6):
                for r4 in range(8):
                    print((r1, r2, r3, r4), end='')
                    print(" : ", end='')
                    print(setPolicy(r1, r2, r3, r4, r1 + r2 + r3 + r4), end='')
                    print(",")